In [ ]:
from flask import Flask, render_template, request, jsonify
import joblib  # or any other library to load your machine learning model
import pickle
import pandas as pd
import numpy as np
from flask_cors import CORS

path = 'clean_data.csv'
data = pd.read_csv(path)
data['PCOS_label'] = None

data = data.set_index('PCOS_label')
data = data.reset_index()

def label(row):
    if row['PCOS'] == 'Yes':
        return 1
    else:
        return 0
    
data['PCOS_label'] = data.apply(lambda row: label(row), axis=1)

PCOS_check = dict(zip(data.PCOS_label.unique(), data.PCOS.unique()))
PCOS_check

filename = 'model.pkl'
loaded_model = pickle.load(open(filename, 'rb'))


feature_cols = ['Period Length', 'Cycle Length', 'Age', 'Overweight', 'loss weight gain / weight loss', 'irregular or missed periods', 'Difficulty in conceiving', 'Hair growth on Chin', 'Hair growth  on Cheeks', 'Hair growth Between breasts',
                'Hair growth  on Upper lips ', 'Hair growth in Arms', 'Hair growth on Inner thighs', 'Acne or skin tags', 'Hair thinning or hair loss ', 'Dark patches', 'always tired', 'more Mood Swings', 'exercise per week', 'eat outside per week', 'canned food often']

app = Flask(__name__)
CORS(app)
# Load your machine learning model
# Replace 'your_model_file.pkl' with the actual filename
model = joblib.load('model.pkl')


@app.route('/predict', methods=['POST'])
def predict():
    data=request.get_json()
    x_test = data['features']
    received_array = np.array(x_test).reshape(1, -1)
    result = loaded_model.predict(received_array)
    result=PCOS_check[result[0]]

    print(received_array)
    return jsonify({'result':result})

if __name__ == '__main__':
    app.run(port=5000)
